In [1]:
# TO DO:
#readme:
    #challenge - # ensure places aren't repeated.  get them dupes out

# do iterative 50 at a time thingy

# markdown

#extra fun:
    # Extract subnational unit ie state where appropriate
        # list as challenge?

    # Print summary rollup of missed cities, and save log to txt file
        # only print summary records to readout here, reference full log file for further details.
        # https://www.pythontutorial.net/python-basics/python-create-text-file/
        # Enable verbose logging heh
    
    # Explore other fun variables to track in provided weather data.

In [11]:
import sys



In [12]:
# Namespace
# Modules
import numpy as np
import pandas as pd
import requests
import string

# import os
from citipy import citipy
from pprint import pprint

# api keys, etc.  Is there a better way to import this than duplicating this in a second spot in the repo?
#https://www.geeksforgeeks.org/python-import-module-from-different-directory/
sys.path.insert(0,'../')
from config import weather_api_key

In [3]:
num_cities = 2000

# generate 2k random lat/long values
lats = np.random.uniform(low=-90.0, high=90.0, size=num_cities)
longs = np.random.uniform(low=-180.0, high=180.0, size=num_cities)
coords_zip = zip(lats,longs)
coords = list(coords_zip)

In [4]:
#instantiate base URL
base_url = "http://api.openweathermap.org/data/2.5/weather"

# instantiate list of results. will be comprised of dictionaries.
weather_data = list()

# instantiate list of full placenames (city plus country code) to eliminate dupes
full_placenames = list()

# Statistics for logging
dupe_count = 0
city_not_found_count = 0

for i in range(200):
    # use citypy to get nearest cities
    city_obj = citipy.nearest_city(latitude=coords[i][0], longitude=coords[i][1])

    # Lower case strings for formal place names bother me.  So we're fixing that.
    city_name = string.capwords(city_obj.city_name)
    country_code = city_obj.country_code.upper()
    
    full_placename = f"{city_name},{country_code}"

    if full_placename not in full_placenames:

        full_placenames.append(full_placename)
        
        params={'appid':weather_api_key,
                'q':full_placename,
                'units':'imperial'
               }

        weather_response = requests.get(url=base_url,params=params)

        try:
            weather_json = weather_response.json()

            weather_data.append({
                "City" : city_name,
                "Country" : country_code,
                "Lat" : weather_json['coord']['lat'],
                "Long" : weather_json['coord']['lon'],
                "Max Temp" : weather_json['main']['temp_max'],
                "Pct Humidity" : weather_json['main']['humidity'],
                "Pct Cloudiness" : weather_json['clouds']['all'],
                "Wind Speed" : weather_json['wind']['speed'],
                "Description" : weather_json['weather'][0]['description']
            })

            print(f"Successfully processed {full_placename}.")
        except KeyError:
            print(f"API call failed to retrieve data for {full_placename}.  Error code: {weather_response.status_code}.")
            city_not_found_count += 1
        
    else:
        print(f"{full_placename} already exists in data set.  Skipping.")
        dupe_count += 1

print("------------------------------------------")
print("Processing complete.  BUT I NEED A TIMESTAMP.")
print(f"Number of coordinates generated: {num_cities}")
print(f"Number of duplicate city records identified: {dupe_count}")
print(f"Total unique cities identified: {len(full_placenames)}")
print("------------------------------------------")
print(f"Number of cities with unidentifiable weather reports: {city_not_found_count}") # REWORD ME
print(f"Successfully processed {len(weather_data)} total records.")

Successfully processed Plettenberg Bay,ZA.
API call failed to retrieve data for Belushya Guba,RU.  Error code: 404.
Successfully processed Atuona,PF.
Successfully processed Novo Aripuana,BR.
Successfully processed Tasiilaq,GL.
Successfully processed Carnarvon,AU.
Successfully processed Lompoc,US.
Successfully processed Araguaina,BR.
Successfully processed Norman Wells,CA.
Successfully processed Izvoarele,RO.
Successfully processed Port Alfred,ZA.
Successfully processed Bonavista,CA.
API call failed to retrieve data for Asayita,ET.  Error code: 404.
Successfully processed Ushuaia,AR.
Successfully processed Hithadhoo,MV.
Successfully processed Haines Junction,CA.
Successfully processed Butaritari,KI.
Successfully processed Arraial Do Cabo,BR.
Carnarvon,AU already exists in data set.  Skipping.
Successfully processed Mount Gambier,AU.
Successfully processed Tehachapi,US.
Successfully processed Qaanaaq,GL.
Successfully processed Vestmannaeyjar,IS.
Successfully processed Punta Arenas,CL.
Su

In [5]:
weather_data_df = pd.DataFrame(weather_data)

weather_data_df.to_csv("WeatherPy_Database.csv")

weather_data_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Plettenberg Bay,ZA,-34.0527,23.3716,65.46,88,19,5.32,few clouds
1,Atuona,PF,-9.8000,-139.0333,77.95,77,11,16.02,few clouds
2,Novo Aripuana,BR,-5.1206,-60.3797,81.16,78,100,2.21,overcast clouds
3,Tasiilaq,GL,65.6145,-37.6368,15.96,55,12,4.34,few clouds
4,Carnarvon,AU,-24.8667,113.6333,75.27,78,0,13.80,clear sky
...,...,...,...,...,...,...,...,...,...
116,Capinota,BO,-17.7167,-66.2333,80.65,29,8,7.00,clear sky
117,Vanimo,PG,-2.6741,141.3028,77.59,91,100,4.54,overcast clouds
118,Bredasdorp,ZA,-34.5322,20.0403,63.72,63,100,11.50,overcast clouds
119,Gornopravdinsk,RU,60.0500,69.9000,1.47,95,100,6.04,overcast clouds
